# 🐱🐶 Cat vs Dog Image Classifier using CNN

## Project Overview
This notebook demonstrates the development of a Convolutional Neural Network (CNN) for binary image classification, distinguishing between cats and dogs. The model achieves **88.66% validation accuracy** on the famous Dogs vs Cats dataset from Kaggle.

## Key Features
- Custom CNN architecture with 4 convolutional blocks
- Data augmentation for improved generalization
- Feature map visualization to understand what the network learns
- Interactive prediction interface

## Table of Contents
1. [Environment Setup](#1-environment-setup)
2. [Data Acquisition](#2-data-acquisition)
3. [Exploratory Data Analysis](#3-exploratory-data-analysis)
4. [Data Preprocessing](#4-data-preprocessing)
5. [Model Architecture](#5-model-architecture)
6. [Training](#6-training)
7. [Model Evaluation](#7-model-evaluation)
8. [Feature Visualization](#8-feature-visualization)
9. [Inference](#9-inference)

## 1. Environment Setup
First, we'll import all necessary libraries and check our computing resources.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
import shutil
import pickle

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Keras version: {keras.__version__}")

## 2. Data Acquisition
We'll download the Dogs vs Cats dataset from Kaggle. This dataset contains 25,000 training images.

In [ ]:
# Install and setup Kaggle API
!pip install -q kaggle

# For Google Colab users: Upload your kaggle.json
# from google.colab import files
# uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q dogs-vs-cats-redux-kernels-edition.zip
!unzip -q train.zip
!unzip -q test.zip

print("✅ Dataset downloaded and extracted successfully!")

## 3. Exploratory Data Analysis
Let's explore our dataset to understand its structure and visualize some samples.

In [ ]:
# Dataset statistics
train_dir = '/content/train'
test_dir = '/content/test'

train_files = os.listdir(train_dir) if os.path.exists(train_dir) else []
test_files = os.listdir(test_dir) if os.path.exists(test_dir) else []

cat_files = [f for f in train_files if f.startswith('cat.')]
dog_files = [f for f in train_files if f.startswith('dog.')]

print("📊 Dataset Statistics:")
print(f"Total training images: {len(train_files)}")
print(f"Total test images: {len(test_files)}")
print(f"Cat images: {len(cat_files)}")
print(f"Dog images: {len(dog_files)}")
print(f"Class balance: {len(cat_files)/len(train_files)*100:.1f}% cats, {len(dog_files)/len(train_files)*100:.1f}% dogs")

In [ ]:
# Visualize random samples
fig, axes = plt.subplots(2, 4, figsize=(15, 8))
fig.suptitle('Dataset Sample: Cats (top) vs Dogs (bottom)', fontsize=16, fontweight='bold')

# Display 4 random cats
for i in range(4):
    if cat_files:
        random_cat = random.choice(cat_files)
        img = mpimg.imread(os.path.join(train_dir, random_cat))
        axes[0, i].imshow(img)
        axes[0, i].set_title(f'Cat #{random_cat.split(".")[1]}')
        axes[0, i].axis('off')

# Display 4 random dogs
for i in range(4):
    if dog_files:
        random_dog = random.choice(dog_files)
        img = mpimg.imread(os.path.join(train_dir, random_dog))
        axes[1, i].imshow(img)
        axes[1, i].set_title(f'Dog #{random_dog.split(".")[1]}')
        axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## 4. Data Preprocessing
We'll organize our data into subdirectories and create data generators with augmentation.

In [ ]:
# Model hyperparameters
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32
EPOCHS = 15
VALIDATION_SPLIT = 0.2

print("🔧 Model Configuration:")
print(f"Input image size: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training epochs: {EPOCHS}")
print(f"Validation split: {VALIDATION_SPLIT*100}%")

In [ ]:
# Organize images into class subdirectories
def organize_dataset(base_dir):
    """Organize images into cats/ and dogs/ subdirectories."""
    cats_dir = os.path.join(base_dir, 'cats')
    dogs_dir = os.path.join(base_dir, 'dogs')
    
    os.makedirs(cats_dir, exist_ok=True)
    os.makedirs(dogs_dir, exist_ok=True)
    
    files = [f for f in os.listdir(base_dir) if f.endswith('.jpg')]
    
    cats_moved = dogs_moved = 0
    
    for filename in files:
        if filename.startswith('cat.'):
            shutil.move(os.path.join(base_dir, filename), 
                       os.path.join(cats_dir, filename))
            cats_moved += 1
        elif filename.startswith('dog.'):
            shutil.move(os.path.join(base_dir, filename), 
                       os.path.join(dogs_dir, filename))
            dogs_moved += 1
    
    return cats_moved, dogs_moved

# Only organize if not already done
if not os.path.exists('/content/train/cats'):
    cats_moved, dogs_moved = organize_dataset('/content/train')
    print(f"✅ Organized {cats_moved} cat images and {dogs_moved} dog images")
else:
    print("✅ Dataset already organized")

In [ ]:
# Create data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    validation_split=VALIDATION_SPLIT
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

print(f"\n📊 Data Generator Summary:")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Class mapping: {train_generator.class_indices}")

## 5. Model Architecture
We'll build a custom CNN with 4 convolutional blocks followed by dense layers.

In [ ]:
def create_cnn_model(input_shape=(150, 150, 3)):
    """
    Create a CNN model for binary classification.
    
    Architecture:
    - 4 Convolutional blocks (Conv2D + MaxPooling2D)
    - Dropout for regularization
    - Dense layers for classification
    """
    model = models.Sequential([
        Input(shape=input_shape),
        
        # Block 1: 32 filters
        layers.Conv2D(32, (3, 3), activation='relu', name='conv1'),
        layers.MaxPooling2D(2, 2, name='pool1'),
        
        # Block 2: 64 filters
        layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
        layers.MaxPooling2D(2, 2, name='pool2'),
        
        # Block 3: 128 filters
        layers.Conv2D(128, (3, 3), activation='relu', name='conv3'),
        layers.MaxPooling2D(2, 2, name='pool3'),
        
        # Block 4: 128 filters
        layers.Conv2D(128, (3, 3), activation='relu', name='conv4'),
        layers.MaxPooling2D(2, 2, name='pool4'),
        
        # Classification head
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu', name='dense1'),
        layers.Dense(1, activation='sigmoid', name='output')
    ])
    
    return model

# Create and compile model
model = create_cnn_model()
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display model architecture
model.summary()

## 6. Training
Train the model with our augmented data.

In [ ]:
# Train the model
print("🚀 Starting training...")
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1
)

print("\n✅ Training completed!")
print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

## 7. Model Evaluation
Visualize training history and save the model.

In [ ]:
# Plot training history
def plot_training_history(history):
    """Plot training and validation accuracy/loss."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy plot
    ax1.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')
    ax1.grid(True, alpha=0.3)
    
    # Loss plot
    ax2.plot(history.history['loss'], label='Training Loss', linewidth=2)
    ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

In [ ]:
# Save model and training history
# For Google Colab
# from google.colab import drive
# drive.mount('/content/drive')
# model.save('/content/drive/MyDrive/cat_dog_cnn_model.h5')

# For local environment
model.save('cat_dog_cnn_model.h5')
print("✅ Model saved successfully!")

# Save training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("✅ Training history saved!")

## 8. Feature Visualization
Visualize what the CNN learns at different layers.

In [ ]:
def visualize_feature_maps(model, img_path, num_layers=8):
    """
    Visualize feature maps from CNN layers.
    Shows what patterns the network detects at different depths.
    """
    # Create a model that outputs intermediate layer activations
    layer_outputs = [layer.output for layer in model.layers[:num_layers]]
    activation_model = Model(inputs=model.input, outputs=layer_outputs)
    
    # Preprocess image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    # Get activations
    activations = activation_model.predict(img_array, verbose=0)
    
    # Plot
    layer_names = ['Conv1', 'Pool1', 'Conv2', 'Pool2', 
                   'Conv3', 'Pool3', 'Conv4', 'Pool4']
    
    fig = plt.figure(figsize=(20, 10))
    
    # Show original
    plt.subplot(2, 5, 1)
    plt.imshow(img)
    plt.title('Original Image', fontweight='bold')
    plt.axis('off')
    
    # Show feature maps
    for i, (activation, name) in enumerate(zip(activations[:8], layer_names)):
        plt.subplot(2, 5, i+2)
        # Display first channel of the activation
        plt.imshow(activation[0, :, :, 0], cmap='viridis')
        plt.title(f'{name}\n{activation.shape[1:3]}', fontsize=10)
        plt.axis('off')
    
    plt.suptitle('CNN Feature Maps Visualization', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Example usage (requires an image path)
# visualize_feature_maps(model, 'path/to/image.jpg')

## 9. Inference
Functions for making predictions on new images.

In [ ]:
def predict_image(model, img_path, show_plot=True):
    """
    Predict whether an image contains a cat or dog.
    
    Args:
        model: Trained Keras model
        img_path: Path to image file
        show_plot: Whether to display the image with prediction
    
    Returns:
        dict: Prediction results with class and confidence
    """
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    # Make prediction
    prediction = model.predict(img_array, verbose=0)
    probability = prediction[0][0]
    
    # Determine class
    if probability > 0.5:
        predicted_class = 'Dog'
        confidence = probability
    else:
        predicted_class = 'Cat'
        confidence = 1 - probability
    
    # Display result
    if show_plot:
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title(f'Prediction: {predicted_class}\nConfidence: {confidence:.2%}', 
                 fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.show()
    
    return {
        'class': predicted_class,
        'confidence': confidence,
        'raw_probability': probability
    }

# Test the model
print("🎯 Model ready for predictions!")
print("Use: predict_image(model, 'path/to/your/image.jpg')")

## Summary

### Model Performance
- **Training Accuracy**: ~87%
- **Validation Accuracy**: ~89%
- **Architecture**: 4-block CNN with 3.45M parameters
- **Training Time**: ~33 minutes on GPU

### Key Techniques Used
1. **Data Augmentation**: Rotation, shifting, flipping, and zooming to prevent overfitting
2. **Dropout Regularization**: 50% dropout before final dense layer
3. **Progressive Feature Extraction**: Increasing filter sizes (32→64→128→128)
4. **Binary Cross-Entropy Loss**: Optimal for two-class classification

### Next Steps
- Deploy model using Streamlit for web interface
- Experiment with transfer learning (VGG16, ResNet50)
- Implement Grad-CAM for better interpretability
- Add model versioning and experiment tracking